### План

1. Разбиваем весь текст на токены-слова
2. Удаляем стоп-слова. Местоимения и артикли в исключения
2. Приводим все к нижнему регистру
3. Уникализируем и подсчитываем количество каждого слова
4. Создаем df
4. К каждому слову добавляем столько раз предложение, сколько это слово встречается.
5. Определяем часть речи и добавляем в df
6. Если слово - глагол, что находим его 3 формы и списком добавляем в df. Проделываем эти операции со всеми частями речи по условию if
Если артикль, то добавляем этим атриклям в df список всех артиклей (для выбора)


### Виды упражнений
1. Определить правильный артикль
2. Определяем пропущенное слово
3. Определить правильную форму глагола. Ищем в предложении все глаголы и предлагаем 3 формы. Не, не пойдет. Времен больше намного
4. Расставить слова в правильном порядке. Перемешиваем все и даем выбрать





In [1]:
import nltk
import streamlit as st
import pandas as pd
import spacy
import pyinflect
import random     
import numpy as np

from tqdm import tqdm

#from googletrans import Translator


#from nltk.corpus import stopwords


nltk.download('punkt')

random.seed(42)


2023-07-03 17:56:32.737724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /Users/Sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Определяем константой типы заданий

TASKS = ['Выберите слово', 'Заполните пропуск']

In [ ]:
# Загружаем каждое предложение в отдельную строку df. Ставим сепоратор таб, иначе запятая выдает ошибку
df = pd.read_csv('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt', lineterminator = '\n', header=None, sep="\t", names=['origin_sentences'])
df.head(5)

In [12]:
with open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt') as f:
    text = f.read()

text = text.replace('\n','')

text = text.replace('"', '')
text = text.replace(',', '')
text = text.replace(':', '')
text = text.replace('-"', '')
text = text.lower()

In [13]:
text

"once upon a time there was a sweet little girl. everyone who saw her liked her but most of all her grandmother who did not know what to give the child next. once she gave her a little cap made of red velvet. because it suited her so well and she wanted to wear it all the time she came to be known as little red cap.one day her mother said to her come little red cap. here is a piece of cake and a bottle of wine. take them to your grandmother. she is sick and weak and they will do her well. mind your manners and give her my greetings. behave yourself on the way and do not leave the path or you might fall down and break the glass and then there will be nothing for your sick grandmother.little red cap promised to obey her mother. the grandmother lived out in the woods a half hour from the village. when little red cap entered the woods a wolf came up to her. she did not know what a wicked animal he was and was not afraid of him.good day to you little red cap.thank you wolf.where are you goi

In [105]:
tokens_sens = nltk.tokenize.sent_tokenize(text, language='english')

#Создаем датафрейм
#df = pd.DataFrame(columns=['origin_sentences', 'tokens'])
df = pd.DataFrame({'origin_sentences': tokens_sens})
df["origin_sentences"] = nltk.tokenize.sent_tokenize(text, language='english')
df["origin_sentences"] = df.apply(lambda x: x['origin_sentences'].replace('.', ''), axis=1)

In [56]:
df

,origin_sentences
0,once upon a time there was a sweet little girl
1,everyone who saw her liked her but most of all...
2,once she gave her a little cap made of red velvet
3,because it suited her so well and she wanted t...
4,here is a piece of cake and a bottle of wine
...,...
62,little red cap carried water until the large l...
63,the smell of sausage arose into the wolf's nose
64,he sniffed and looked down stretching his neck...
65,he slid off the roof fell into the trough and ...


In [52]:
# for index, sentence in df.iterrows():
#     #df['tokens'][index] = [token for token in nlp(sentence)]
#     #print(token)
#     df['tokens'][index] = [token for token in nlp(str(sentence))]

#     #df.loc[len(df)]=[sentence, options, answer, task, []]  
# df

,origin_sentences,tokens
0,once upon a time there was a sweet little girl,"[origin_sentences, , once, upon, a, time, t..."
1,everyone who saw her liked her but most of all...,"[origin_sentences, , everyone, who, saw, he..."
2,once she gave her a little cap made of red velvet,"[origin_sentences, , once, she, gave, her, ..."
3,because it suited her so well and she wanted t...,"[origin_sentences, , because, it, suited, h..."
4,here is a piece of cake and a bottle of wine,"[origin_sentences, , here, is, a, piece, of..."
...,...,...
62,little red cap carried water until the large l...,"[origin_sentences, , little, red, cap, carr..."
63,the smell of sausage arose into the wolf's nose,"[origin_sentences, , the, smell, of, sausag..."
64,he sniffed and looked down stretching his neck...,"[origin_sentences, , he, sniffed, and, look..."
65,he slid off the roof fell into the trough and ...,"[origin_sentences, , he, slid, off, the, ro..."


In [106]:
tokens = []
lemma = []
pos = []

for doc in nlp.pipe(df['origin_sentences'].astype('unicode').values, batch_size=50,
                        ):
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
        
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

df['word_tokens'] = tokens
df['word_lemma'] = lemma
df['word_pos'] = pos

/var/folders/3s/0vl43zsj2n79hlj09xb7sby40000gn/T/ipykernel_32651/3081040105.py:7: DeprecationWarning: [W107] The property `Doc.is_parsed` is deprecated. Use `Doc.has_annotation("DEP")` instead.
  if doc.is_parsed:


In [58]:
df

,origin_sentences,word_tokens,word_lemma,word_pos
0,once upon a time there was a sweet little girl,"[once, upon, a, time, there, was, a, sweet, li...","[once, upon, a, time, there, be, a, sweet, lit...","[ADV, SCONJ, DET, NOUN, PRON, VERB, DET, ADJ, ..."
1,everyone who saw her liked her but most of all...,"[everyone, who, saw, her, liked, her, but, mos...","[everyone, who, see, she, like, she, but, most...","[PRON, PRON, VERB, PRON, VERB, PRON, CCONJ, AD..."
2,once she gave her a little cap made of red velvet,"[once, she, gave, her, a, little, cap, made, o...","[once, she, give, she, a, little, cap, make, o...","[SCONJ, PRON, VERB, PRON, DET, ADJ, NOUN, VERB..."
3,because it suited her so well and she wanted t...,"[because, it, suited, her, so, well, and, she,...","[because, it, suit, she, so, well, and, she, w...","[SCONJ, PRON, VERB, PRON, ADV, ADV, CCONJ, PRO..."
4,here is a piece of cake and a bottle of wine,"[here, is, a, piece, of, cake, and, a, bottle,...","[here, be, a, piece, of, cake, and, a, bottle,...","[ADV, AUX, DET, NOUN, ADP, NOUN, CCONJ, DET, N..."
...,...,...,...,...
62,little red cap carried water until the large l...,"[little, red, cap, carried, water, until, the,...","[little, red, cap, carry, water, until, the, l...","[ADJ, ADJ, NOUN, VERB, NOUN, SCONJ, DET, ADJ, ..."
63,the smell of sausage arose into the wolf's nose,"[the, smell, of, sausage, arose, into, the, wo...","[the, smell, of, sausage, arise, into, the, wo...","[DET, NOUN, ADP, NOUN, VERB, ADP, DET, NOUN, P..."
64,he sniffed and looked down stretching his neck...,"[he, sniffed, and, looked, down, stretching, h...","[he, sniff, and, look, down, stretch, his, nec...","[PRON, VERB, CCONJ, VERB, ADP, VERB, PRON, NOU..."
65,he slid off the roof fell into the trough and ...,"[he, slid, off, the, roof, fell, into, the, tr...","[he, slide, off, the, roof, fall, into, the, t...","[PRON, VERB, ADP, DET, NOUN, VERB, ADP, DET, N..."


In [107]:
df

,origin_sentences,word_tokens,word_lemma,word_pos
0,once upon a time there was a sweet little girl,"[once, upon, a, time, there, was, a, sweet, li...","[once, upon, a, time, there, be, a, sweet, lit...","[ADV, SCONJ, DET, NOUN, PRON, VERB, DET, ADJ, ..."
1,everyone who saw her liked her but most of all...,"[everyone, who, saw, her, liked, her, but, mos...","[everyone, who, see, she, like, she, but, most...","[PRON, PRON, VERB, PRON, VERB, PRON, CCONJ, AD..."
2,once she gave her a little cap made of red velvet,"[once, she, gave, her, a, little, cap, made, o...","[once, she, give, she, a, little, cap, make, o...","[SCONJ, PRON, VERB, PRON, DET, ADJ, NOUN, VERB..."
3,because it suited her so well and she wanted t...,"[because, it, suited, her, so, well, and, she,...","[because, it, suit, she, so, well, and, she, w...","[SCONJ, PRON, VERB, PRON, ADV, ADV, CCONJ, PRO..."
4,here is a piece of cake and a bottle of wine,"[here, is, a, piece, of, cake, and, a, bottle,...","[here, be, a, piece, of, cake, and, a, bottle,...","[ADV, AUX, DET, NOUN, ADP, NOUN, CCONJ, DET, N..."
...,...,...,...,...
62,little red cap carried water until the large l...,"[little, red, cap, carried, water, until, the,...","[little, red, cap, carry, water, until, the, l...","[ADJ, ADJ, NOUN, VERB, NOUN, SCONJ, DET, ADJ, ..."
63,the smell of sausage arose into the wolf's nose,"[the, smell, of, sausage, arose, into, the, wo...","[the, smell, of, sausage, arise, into, the, wo...","[DET, NOUN, ADP, NOUN, VERB, ADP, DET, NOUN, P..."
64,he sniffed and looked down stretching his neck...,"[he, sniffed, and, looked, down, stretching, h...","[he, sniff, and, look, down, stretch, his, nec...","[PRON, VERB, CCONJ, VERB, ADP, VERB, PRON, NOU..."
65,he slid off the roof fell into the trough and ...,"[he, slid, off, the, roof, fell, into, the, tr...","[he, slide, off, the, roof, fall, into, the, t...","[PRON, VERB, ADP, DET, NOUN, VERB, ADP, DET, N..."


['everyone', 'who', 'saw', 'her', 'liked', 'her', 'but', 'most', 'of', 'all', 'her', 'grandmother', 'who', 'did', 'not', 'know', 'what', 'to', 'give', 'the', 'child', 'next']
[True, False, False]


In [187]:
# Записываем флаг на упражнения с артиклем
df['is_article'] = df['word_tokens'].apply(lambda x: any([i in x for i in ['the', 'a', 'an']])) 
df



,origin_sentences,word_tokens,word_lemma,word_pos,is_article
0,once upon a time there was a sweet little girl,"[once, upon, a, time, there, was, a, sweet, li...","[once, upon, a, time, there, be, a, sweet, lit...","[ADV, SCONJ, DET, NOUN, PRON, VERB, DET, ADJ, ...",True
1,everyone who saw her liked her but most of all...,"[everyone, who, saw, her, liked, her, but, mos...","[everyone, who, see, she, like, she, but, most...","[PRON, PRON, VERB, PRON, VERB, PRON, CCONJ, AD...",True
2,once she gave her a little cap made of red velvet,"[once, she, gave, her, a, little, cap, made, o...","[once, she, give, she, a, little, cap, make, o...","[SCONJ, PRON, VERB, PRON, DET, ADJ, NOUN, VERB...",True
3,because it suited her so well and she wanted t...,"[because, it, suited, her, so, well, and, she,...","[because, it, suit, she, so, well, and, she, w...","[SCONJ, PRON, VERB, PRON, ADV, ADV, CCONJ, PRO...",True
4,here is a piece of cake and a bottle of wine,"[here, is, a, piece, of, cake, and, a, bottle,...","[here, be, a, piece, of, cake, and, a, bottle,...","[ADV, AUX, DET, NOUN, ADP, NOUN, CCONJ, DET, N...",True
...,...,...,...,...,...
62,little red cap carried water until the large l...,"[little, red, cap, carried, water, until, the,...","[little, red, cap, carry, water, until, the, l...","[ADJ, ADJ, NOUN, VERB, NOUN, SCONJ, DET, ADJ, ...",True
63,the smell of sausage arose into the wolf's nose,"[the, smell, of, sausage, arose, into, the, wo...","[the, smell, of, sausage, arise, into, the, wo...","[DET, NOUN, ADP, NOUN, VERB, ADP, DET, NOUN, P...",True
64,he sniffed and looked down stretching his neck...,"[he, sniffed, and, looked, down, stretching, h...","[he, sniff, and, look, down, stretch, his, nec...","[PRON, VERB, CCONJ, VERB, ADP, VERB, PRON, NOU...",False
65,he slid off the roof fell into the trough and ...,"[he, slid, off, the, roof, fell, into, the, tr...","[he, slide, off, the, roof, fall, into, the, t...","[PRON, VERB, ADP, DET, NOUN, VERB, ADP, DET, N...",True


In [218]:
s1 = pd.Series([12.539048, 5.688863, 16.247349, 9.601944, 5.551343, 5.591008, 10.093195, 11.189223, 10.067743, 1.822353])

arr = [s1.min(), s1.quantile(0.25), s1.median(), s1.quantile(0.75), s1.max()]
pd.Series(arr)
s1.describe().T

s = pd.Series(list('abdcbdhafceeabcahbcaedadaecbbbecafgh'))
aa = s.value_counts().sort_values(ascending=False)[0:2].index
s.where(s.isin(aa), 'Yet another value')

s1 = pd.Series(range(0, 10))
#print(pd.DataFrame(data=[[_ for _ in s1[:5]], [_ for _ in s1[5:]]])) 

s1 = pd.Series([5, 10, 2, 3, 9, 12, 1])
s1 = s1.to_numpy()
np.argwhere(s1%2==0)
#np.argwhere(s1)



array([[1],
       [2],
       [5]])

In [ ]:
df.head(5)

In [59]:
import pyinflect
nlp = spacy.load('en_core_web_sm')
tokens = nlp('This is an example of xxtest.')
tokens[3]._.inflect('NNS')

'examples'

In [ ]:
# изменение степени прилагательного с помощью pyinflect
for token in nlp("I think it's a good idea and easy to use"):
    if word_pos=='ADJ':
        print(token.text, token._.inflect('JJS'))        

In [60]:
item = 'ADJ'
for i in df.word_pos:
   
    if item in i:
        print(item)

ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ
ADJ


In [ ]:
if 'ADJ'.isin(df.word_pos):
    print('ok')

In [ ]:


df["ADJ_forms"] = df.apply(lambda x: int(x["ride_cost"] * 0.95)
                                 if x["rating"] > 6 and x["speed_max"] < 120
                                 else int(x["ride_cost"] * 1.05),
                                 axis=1)

In [ ]:
for word in df_all_words.origin_word:
    print('lemma:', word.lemma_)

In [ ]:
# Приводим слова к начальной форме (лемме)
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(df['origin_sentences'].astype('unicode').values)

token = doc
print(token.text)
for word in doc:
    print('lemma:', word.lemma_)



### NLTK

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
stops = set(stopwords.words('english'))
print(stops)

In [ ]:
# Токенизируем каждо

df['bag_of_words'] = df.origin_sentences.apply(lambda x: nltk.word_tokenize(x))

#result2 = df.assign(feature_1=df.apply(nltk.word_tokenize("a house word ")))

#bag_of_words

In [ ]:
df

In [ ]:
df_words = pd.DataFrame({'origin_words':bag_of_words})
df_words

In [ ]:
# Этот код ищет словоформы, но только существвительные

from nltk.corpus import wordnet as wn

forms = set() #We'll store the derivational forms in a set to eliminate duplicates
for happy_lemma in wn.lemmas("like"): #for each "happy" lemma in WordNet
    forms.add(happy_lemma.name()) #add the lemma itself
    for related_lemma in happy_lemma.derivationally_related_forms(): #for each related lemma
        forms.add(related_lemma.name()) #add the related lemma
forms        

In [ ]:
tokens_sens = nltk.tokenize.sent_tokenize(text, language='english')

In [ ]:
pd.set_option('display.max_colwidth', None)
df_sentences = pd.DataFrame({'sentence': tokens_sens})
df_sentences

In [ ]:
pd.set_option('display.max_colwidth', 50)

### Spacy

In [ ]:
# @Language.component("segm")
# def set_custom_segmentation(doc):
#     for token in doc[:-1]:
#         if token.text == '{S}':
#             doc[token.i+1].is_sent_start = True
#         else:
#             doc[token.i+1].is_sent_start = False
#     return doc

In [ ]:
import spacy
#from spacy.lang.ru import Russian
#from spacy import displacy
#from spacy.lang.en.examples import sentences 

#import en_core_web_sm
#nlp = en_core_web_sm.load()

nlp = spacy.load("en_core_web_sm")
doc = nlp('''
Once upon a time there was a sweet little girl. Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next. Once she gave her a little cap made of red velvet. Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Cap.
One day her mother said to her, "Come Little Red Cap. Here is a piece of cake and a bottle of wine. Take them to your grandmother. She is sick and weak, and they will do her well. Mind your manners and give her my greetings. Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother."

Little Red Cap promised to obey her mother. The grandmother lived out in the woods, a half hour from the village. When Little Red Cap entered the woods a wolf came up to her. She did not know what a wicked animal he was, and was not afraid of him.

"Good day to you, Little Red Cap."

"Thank you, wolf."

"Where are you going so early, Little Red Cap?"

"To grandmother's."

"And what are you carrying under your apron?"
''')
#open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt', encoding ='utf-8')
i=0
for sent in doc.sents:
    i+=1
    print('i=', str(i)+sent.text)

In [ ]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
token = doc
print(token.text)
for word in doc:
    print('lemma:', word.lemma_)

In [ ]:
nlp = Russian()
doc = nlp("Съешь ещё этих мягких французских булок, да выпей чаю.")

In [ ]:
token = doc[0]
print(token.text)

span = doc[3:6]
print(span.text)

In [ ]:
print("is_alpha:    ", [token.is_alpha for token in doc])
print("is_punct:    ", [token.is_punct for token in doc])
print("like_num:    ", [token.like_num for token in doc])

### Gensim

In [ ]:
import gensim
import os
from gensim.utils import simple_preprocess
  
#  прочитать текстовый файл как объект
doc = open('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt', encoding ='utf-8')
  
# предварительная обработка файла для получения списка токенов
tokenized = []
for sentence in doc.read().split('.'):
    # функция simple_preprocess возвращает список слов каждого предложения
    tokenized.append(simple_preprocess(sentence, deacc = True))
  
for i in tokenized:
    print(i)
doc.close()

### Pyinflect чтобы изменять слова (времена глагола, степени прилагательных, число множ/ед)

In [ ]:
import pyinflect
nlp = spacy.load('en_core_web_sm')
tokens = nlp('This is an example of xxtest.')
tokens[3]._.inflect('NNS')

In [ ]:
for i in range(len(tokens)):
    # тут условие if проверка на часть речи
    for j in range(0, 3):
        print(tokens[i]._.inflect('VBD', form_num=j))

In [ ]:
tokens[1]._.inflect('VBD', form_num=1)

In [ ]:
tokens[5]._.inflect('VBG', inflect_oov=True)

### Bag of word (BOW)

In [ ]:
df_sentences

In [ ]:
allsentences = ["Joe waited for the train train", "The train was late", "Mary and Samantha took the bus"]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(allsentences)
print(X.toarray())

In [ ]:
df_sentences.sentence.values.tolist()

In [ ]:
X